### CO-WIN VACCINE SLOTS AVAILABILITY NOTIFICATION APP

In [2]:
import requests
import datetime
import json
import warnings

import smtplib
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
warnings.filterwarnings("ignore")
import json2table


def call(POST_CODE):
    numdays = 7
    abc=[]   
    base = datetime.datetime.today()
    date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
    date_str = [x.strftime("%d-%m-%Y") for x in date_list]
    headers= {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    for PC in POST_CODE:
            for INP_DATE in date_str:
                URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={}&date={}".format(PC, INP_DATE)
                response = requests.get(URL,headers=headers,verify=False )
                if not response.ok:
                    headers= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
                    response = requests.get(URL,headers=headers,verify=False )
                if response.ok:
                        resp_json = response.json()

                        if resp_json["centers"]:
                            with open("resp_{}_file.txt".format(INP_DATE), 'w') as outfile:
                                json.dump(resp_json, outfile)  ## save the response file to local for refrence

                            for center in resp_json["centers"]:
                                    center['sessions']= [sn for sn in center['sessions'] if sn['available_capacity']>0 ]
                                    if  center['sessions']:
                                        abc.append(center)    
    if len(abc) ==0:
            return
    else:    
            body=json2table.convert(dict(zip([1,2,3,4,5,6,7,8,9], (abc))), 
                                     build_direction="LEFT_TO_RIGHT", 
                                     table_attributes={"style": "width:100%"})    
    return body

def send_mail(body,codes,rec):

    message = MIMEMultipart()
    message['Subject'] = 'Available vaccines slots for:' + str(codes)    ## Add custom message here
    message['From'] = 'notifications@gmail.com'     ## From Email account
    message['To'] =   rec     ### list of reciepients
    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(message['From'], '1@3$5^7*')    ## From account password
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()
    print("Mail sent successfully to :",rec )
    return 

def vaccine(codes,rec):
    text=call(codes)
    if text:
        for r in rec:
            send_mail(text,codes,r)
    else:
        print("Empty output")
    return    

## Call function  
code_delhi=["122001","110075","110077","110061","110043"]    ### pass a list of pincodes to be call in the API
mail_delhi=[ "abc@gmail.com","bcd@gmail.com"]    ## List of email id to be passed for notification 
vaccine(code_delhi,mail_delhi)   ### Call the API

